In [ ]:
import mediapipe as mp
import math
import cv2
import time
import numpy as np
WIDTH, HEIGHT = 800, 600

#better for depth functions, since less accurate
def distance_3d(p1, p2):
    dx = p1.x - p2.x
    dy = p1.y - p2.y
    dz = p1.z - p2.z
    return math.sqrt(dx**2 + dy**2 + dz**2)
#better for 2D related functions 
def distance_2d_px(p1, p2, w, h):
    x1, y1 = int(p1.x * w), int(p1.y * h)
    x2, y2 = int(p2.x * w), int(p2.y * h)
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

def landmark_to_pixel(landmark, img):
    h, w, _ = img.shape
    x_px = int(landmark.x * w)
    y_px = int(landmark.y * h)
    return (x_px, y_px)

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            pinkymcp = hand_landmarks.landmark[17]
            indexmcp = hand_landmarks.landmark[5]
            wrist = hand_landmarks.landmark[0]
            middlemcp = hand_landmarks.landmark[9]
            hand_width = distance_2d_px(indexmcp, pinkymcp,WIDTH,HEIGHT )
            hand_height = distance_2d_px(wrist,middlemcp,WIDTH,HEIGHT)
            #calculating hand size so we can adjust for different sized hands
            hand_size = (hand_height + hand_width)/2
            
            #finding Euclidian distance for a more accurate result
            #index_tip = hand_landmarks.landmark[8]
           # h, w, _ = img.shape
           # x = int(index_tip.x * w)
           # y = int(index_tip.y * h)
            #dx = hand_landmarks.landmark[8].x - hand_landmarks.landmark[4].x 
            distance = distance_2d_px(hand_landmarks.landmark[8], hand_landmarks.landmark[4],WIDTH,HEIGHT)
            if hand_size > 1e-5: 
                real_distance = distance/hand_size
                print(real_distance)
                
            else:
                real_distance = distance
                


                # Only change color if condition is met AND at least 1 second has passed
            if (real_distance) < 0.003:
                current_time = time.time()


            # Optional: draw hand landmarks
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            

    cv2.imshow("pinching", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
import pygame
import random
import numpy
# Initialize Pygame
pygame.init()

# Screen settings
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Drag the Green Circles into the Black Square")

# Colors
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)

# Fonts
font = pygame.font.SysFont(None, 36)

# Game objects
NUM_CIRCLES = 10
CIRCLE_RADIUS = 15
SQUARE_SIZE = 80
SQUARE_POS = (WIDTH - SQUARE_SIZE - 50, HEIGHT//2 - SQUARE_SIZE//2)

# Generate random circle positions
circles = []
for _ in range(NUM_CIRCLES):
    x = random.randint(CIRCLE_RADIUS, WIDTH - CIRCLE_RADIUS - SQUARE_SIZE - 100)
    y = random.randint(CIRCLE_RADIUS, HEIGHT - CIRCLE_RADIUS)
    circles.append({'pos': [x, y], 'dragging': False})

# Score
score = 0

# Main loop
running = True
clock = pygame.time.Clock()

while running:
    screen.fill(WHITE)

    # Draw black square
    pygame.draw.rect(screen, BLACK, (*SQUARE_POS, SQUARE_SIZE, SQUARE_SIZE))

    # Draw score
    score_text = font.render(f"Score: {score}", True, BLACK)
    screen.blit(score_text, (WIDTH - 180, 20))

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            for circle in circles:
                cx, cy = circle['pos']
                if (pygame.math.Vector2(pygame.mouse.get_pos()) - pygame.math.Vector2(cx, cy)).length() <= CIRCLE_RADIUS:
                    circle['dragging'] = True
        elif event.type == pygame.MOUSEBUTTONUP:
            for circle in circles:
                if circle['dragging']:
                    circle['dragging'] = False
                    cx, cy = circle['pos']
                    sx, sy = SQUARE_POS
                    if sx <= cx <= sx + SQUARE_SIZE and sy <= cy <= sy + SQUARE_SIZE:
                        circles.remove(circle)
                        score += 1
        elif event.type == pygame.MOUSEMOTION:
            for circle in circles:
                if circle['dragging']:
                    circle['pos'] = list(event.pos)

    # Draw circles
    for circle in circles:
        pygame.draw.circle(screen, GREEN, circle['pos'], CIRCLE_RADIUS)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


pygame 2.3.0 (SDL 2.24.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# Game updates

# use mediapipe to speer a 'gripper' from the right and left hand
# some circles are larger and harder to drag
# click first to reval the circle size, 
# the number of clicks is listed on the circle 

# the game instructs how to pinch the circle to drag it
# haptic feedback when the circle is successfully placed in the square
# some circles need two hands to drag




: 